<font size="+3"><center>
    CCATpHive Documentation
</center></font>

<center><b>James Burgoyne</b></center>
<center>jburgoyne@phas.ubc.ca</center>
<center>June 2022</center>

# Introduction  <a name="introduction"></a>

CCATpHive is python software to mediate communication between the control server and the parallel operating RFSoC boards which control readout operations for the CCAT Prime telescope. Development was begun in January 2022. The software's main goals are: (1) to provide interface hooks (and a terminal interface) to board functionality; (2) to integrate with a Redis server to communicate over a network between the control server and the boards; (3) to provide control of the main readout functionality scripts. 

## Current Status (August 2022)

As of June 2022, a basic working version which accomplishes the 3 goals was complete including the ability to send commands from the control server to the boards via Redis and receive a return string, logging, basic configuration, and module integration to allow for board functionality. 

In July and August additional functionality was integrated in to provide command arguments, return payloads, separation of the board into 4 drones (each controlling an RF network), key/value pairs, and a proof of concept GUI. 

As of September 1st, the single channel module (single_chan) has rudimentary versions of the following functions: writeVnaComb, writeTestTone, getAdcData, getSnapData, vnaSweep, findResonators.

Future work will focus on continuing to add board functionality modules.

# Environment <a name="environment"></a>

For development and testing, the following environments were used:

- MacOS 12.4
- Python 3.8.13
- Redis server 5.0.3

# File Structure <a name="file_structure"></a>

- **alcove_commands/**: Modules containing the functions to perform board tasks.
    - **board_utilities.py**: Basic board utility tools.
    - **init.py**: Initialization script. Must be run to initialize the board.
    - **single_chan.py**: Gateware functions (single channel).
    - **test_functions.py**: Testing functions.
- **docs/**: Documentation and guides.
- **drones/**: The board runs four drones which each have a subdirectory in here.
    - **drone1/**: Configuration and data files specific to drone 1.
        - **_cfg_drone1.py**: Drone 1 specific configuration options.
- **queen_commands/**: Commands which run on the control server instead of the boards.
    - **test_functions**: Testing functions.
- **RFnetworks/**: Contains a subdirectory for each RF network this board has used. Each subdirectory contains configuration and data files specific to that RF network.
- **_cfg_board.py**: RFSoC board configuration options.
- **_cfg_queen.py**: Control server configuration options.
- **alcove_tui.py**: A terminal interface to alcove.py. This is used only to directly interact with alcove.py when locally on the board.
- **alcove.py**: Provides an API to the board functionality functions (commands).
- **drone.py**: Runs on each of the boards (4 instances) and listens for commands from the control server (via Redis). Upon receiving a command it asks alcove.py to execute it and publishes any necessary messages, e.g. the return statement. Should be running at all times to hear commands.
- **queen_gui.ipynb**: Proof of concept graphical interface to queen.py running in a Jupyter notebook.
- **queen_tui.py**: A terminal interface to queen.py.
- **queen.py**: Runs on the command server to publish commands (via Redis) to remote boards, and to listen for messages from the boards. Should be running at all times in listen mode to pick up board messages.
- **quickDataViewer.ipynb**: A simple Jupyter notebook to inspect data in tmp/ (which are payloads from the board functions).

# Redis Channel Structure <a name="channel_structure"></a>

- **all_boards**: Channel to send commands to all boards at once.
- **board_[bid]_[uuid]**: Each board has it's own command channels. A new channel is created every time a command is issued with a random uuid generated string suffix. [bid] is the board identifier (contained in _cfg_board.py) and [cid] is the command identifier which is a unique id generated when the command is sent.
- **rets_board_[bid]_[uuid]**: Each board has it's own return channels, similar to the command channels.
- **board_[bid].[drid]_[uuid]**: Each drone has it's own command channels. A new channel is created every time a command is issued with a random uuid generated string suffix.
- **rets_board_[bid].[drid]_[uuid]**: Each drone has it's own return channels, similar to the command channels.

**[bid]**: Board identifier (contained in _cfg_board.py).  
**[drid]**: Drone identifier (1-4) (contained in _cfg_drone.py).  
**[cid]**: Command identifier which is a unique id generated when the command is sent.  

# Basic Usage <a name="basic_usage"></a>

**Initialization**:

1. Redis server startup (on control server).  
1. Listening Queen startup (on control server).  
1. Run init.py script for each drone.
1. Drone startup on boards: 4 drones per board.

**Basic command flow**:

1. Commands sent to Queen (e.g. via queen_tui.py) are communicated to redis in appropriate channels.
1. Drones are subscribed to redis and execute commands they receive.
1. Drones send return payloads to redis.
1. Listening Queen receives return values and saves/logs them.

## Redis Server
A Redis server must be running for the control server and boards to communicate. The configration options (e.g. host, port, etc.) for the Redis server are in ```_cfg_board.py``` for the boards, and in ```queen.py``` for the control server. The Redis server will be hosted on the control server.

**Start Redis server:**
```bash
redis-server &
```
or to use a custom redis configuration file (to specify IP address and port, for example):
```bash
redis-server /usr/local/etc/redis.conf
```

**Stop Redis server:**  
CTRL-c from the terminal that started the redis server (unless it was started in the background using, e.g. using an ampersand after the start command).  
From an alternate terminal:
```bash
redis-cli shutdown
```

<figure>
  <img src="redis_startup.png" alt="Redis startup in the terminal."/>
  <figcaption>Redis server startup from a terminal. Note that if the ampersand wasn't addended, another terminal would be needed to shutdown the server.</figcaption>
</figure>

## Drone

Each board runs 4 drones (drone.py) in parallel (with configuration and data separated into 4 subdirectories), each operating a separate 512 MHz bandwidth microwave network (divided into 1000 channels). Each drone operates in a 'listening' mode in order to implement commands received from the queen over redis. The drones must be running at all times, and should be started at board startup, and up-status monitored via a daemon.

To start a single instance of drone.py with drone identifier of drid=1 (in listening mode):
```
python drone.py 1
```

## Queen (TUI)

Queen (queen.py) can potentially be interacted with from different user-interfaces, but at time-of-writing the only interface is a terminal-user-interface (queen_tui.py). A proof of concept GUI is in development.

### Help

To get **help** with queen_tui and to see a list of all possible commands:
```
python queen_tui.py -h
```

### Commands

#### Command Number

All commands have an associated command number which is given as the first required positional argument. For example, to send the command with command number 1 (to all drones):  
```
python queen_tui.py 1
```

#### Command Recipient

A command can be for all drones, a specific board, a specific drone, or queen itself. If no recipient for a command is specified, it is assumed to be for all drones on all boards. 

To specify a command for a specific board or drone, use the second positional argument. The format is ```[bid]``` or ```[bid].[drid]```. For example, to send the command 1 to drone 3 on board 2:  
```
python queen_tui.py 1 2.3
```

To specify a command is for queen itself use the ```-q``` or ```--queen``` argument.

#### Command Returns

When a command is sent to a single drone, queen will output the return data. The form of the output data depends on it's type:
- **string**: The string is printed to the terminal. 
- **Numpy array**: The data is saved to a _.npy_ file in _tmp/_. This can be loaded using ```np.load()```.
- **Other types**: The data is written to an extensionless file in _tmp/_. This can be loaded using ```pickle.loads()```.

When a command is sent to multiple drones, the queen script used to send the command doesn't attempt to intercept the return. Instead, if an instance of queen is operating in listen mode, it will intercept all returns and save them to the _tmp/_ folder, as per the above scheme.

# Queen Commands <a name="queen_commands"></a>

## Listen Mode

An instance of queen (queen.py) should be running in 'listen' mode at all times to process board messages. 
To start a single instance of queen.py in **listen mode**:
```
python queen_tui.py 21 -q
```

# Board Commands <a name="board_commands"></a>

Board commands are still evolving rapidly and this section will be filled in in the future.